### with this coding journey to the sound by ML we can answer this question "what is this sound" by its waveform or mel spectrogram picture with the help of CNN and ResNet50 by some tricks for model enhacement

**Dataset** : ESC-50 (environmental sound classification) with 50 classes(e.g., barking, snoring, ...)

**Libraryies** : torchaudio with two other pieces of software: SoX and LibROSA

**models** : 
1. CNN (convolutional Neural Network) using ID variants 
2. pretrained ResNet50 for Mel Spectrogram

**specials** : 
1. find_lr() function to find a decent learning rate better than GridSearch 
2. frequency masking (in the future)
3. time masking (in the future)

In [5]:
# Data
curl https://github.com/karoldvl/Esc-50/archive/master.zip
import Ipython.display as display
display.Audio('Esc-50/audio/1-100032-A-0.wav')

SyntaxError: invalid syntax (2481897942.py, line 2)

In [ ]:
import glob
from collections import Counter

list = [f.split("-")[-1].replace(".wav","") for f in glob.glob("ESC-50/audio/*.wav")]
Counter(list) # convert to dic

In [ ]:
!pip install sox torchaudio

In [ ]:
# building an ESC-50 dataset

class ESC50(Dataset):
    def __init__(self, path):
        files = path(path).glob("*.wav")
        # iterate through the listing and create tuple (filename,label)
        self.items = [f,f.split("-")[-1].replace(".wav","") for f in files]
        self.length = len(self.items)

    def __getitem__(self,index):
        filename, label = self.items[index]
        audio_tensor, sample_rate = torchaudio.load(filename)
        return audio_tensor, label

    def __len__(self):
        return self.length

test_esc50 = ESC50("mypath")
tensor, label = list(test[0])

tensor
tensor.shape

In [ ]:
# shuffle and split data into 60/20/20
first number present 20 percent of dataset so :
1 2 3 for train
4 for valid
5 for test

from pathlib import Path

bs = 64
PATH_TO_ESC-50 = Path.cwd() / 'esc50'
path = 'test.md'
test

train = Path(PATH_TO_ESC-50/ "train")
valid = Path(PATH_TO_ESC-50/ "valid")
test = Path(PATH_TO_ESC-50/ "test")

train_loader = torch.utils.DataLoader(train, batch_size = bs, shuffle = true)
valid_loader = torch.utils.DataLoader(valid, batch_size = bs, shuffle = true)
test_loader = torch.utils.DataLoader(test, batch_size = bs, shuffle = true)

In [ ]:
# CNN model for dataset

class AudioNet(nn.Module):
    def __init__(self):
        super(AudioNet, self).__init__()
        self.conv1 = nn.Conv1d(1, 128, 80, 4)
        self.bn1 = nn.BatchNorm1d(128)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(128, 128, 3)
        self.bn2 = nn.BatchNorm1d(128)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(128, 256, 3)
        self.bn3 = nn.BatchNorm1d(256)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(256, 512, 3)
        self.bn4 = nn.BatchNorm1d(512)
        self.pool4 = nn.MaxPool1d(4)
        self.avgPool = nn.AvgPool(30)
        self.fc1 = nn.Linear(512,10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = self.avgPool(x)
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        return F.log_softmax(x, dim=2)


In [ ]:
audionet = AudioNet()
audionet.to(device)

In [ ]:
# find_lr() function
import math
def find_lr(model, loss_fn, optimizer, init_value=1e-8, final_value=10.0):
number_in_epoch = len(train_loader) - 1
update_step = (final_value/init_value) ** (1/number_in_epoch)
lr = init_value
optimizer.param_groups[0]["lr"] = lr
best_loss = 0.0
batch_num = 0
losses = []
log_lrs = []
for data in train_loader:
    batch_num += 1
    inputs, labels = data
    inputs, labels = inputs, labels
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = loss_fn(outputs, labels)

    # crash out if loss explodes
    if batch_num > 1 and loss > 4 * best_loss:
        return log_lrs[10:-5], losses[10:-5]

    # record the best loss
    if batch_num == 1 or loss < best_loss:
        return log_lrs[10:-5], losses[10:-5]

    # store the value
    losses.append(loss)
    log_lrs.append(math.log10(lr))
    
    # do backward pass and optimize
    loss.backward()
    optimizer.step()

    # update the lr for the next step
    lr *= update_step
    optimizer.param_groups[0]["lr"] = lr
return log_lrs[10:-5], losses[10:-5]


In [ ]:
audionet.save("audionet.pth")
import torch.optim as optim
optimizer = optim.Adam(audionet.parameters(), lr=0.001)
logs, losses = find_lr(audionet, nn.CrossEntropyLoss(), optimizer)
plt.plot(logs,losses)

In [ ]:
# we train model for 20 epochs
# expect to be 13% better than 3% by the chance
train(audionet, optimizer, torch.nn.CrossEntropyLoss(), train_data_loader, valid_data_loader, epochs=20)

lets investigate a way of looking at our audio data that may yield better results, maybe by transfer learning with ResNet50 model 

In [ ]:
sample, sr = libsora.load("ESC-50/train/1-100032-A-0.wav", sr=None)
spectrogram = libsora.feature.melspectrogram(sample, sr=sr)
spectrogram

In [ ]:
libsora.display.specshow(spectrogram, sr=sr, x_axis='time', y_axis='mel')

In [ ]:
# covert to logarithmic for better scale
log_spectrogram = libsora.power_to_db(spectrogram, ref=np.max)

In [ ]:
# a new dataset

class ESC50Spectrogram(Dataset):
    def __init__(self, path):
        files = path(path).glob("*.wav")
        # iterate through the listing and create tuple (filename,label)
        self.items = [f,f.split("-")[-1].replace(".wav","") for f in files]
        self.length = len(self.items)
        self.transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

    def __getitem__(self,index):
        filename, label = self.items[index]
        audio_tensor, sample_rate = libsora.load(filename, sr=None)
        spectrogram = libsora.feature.melspectrogram(audio_tensor, sr=sample_rate)
        log_spectrogram = libsora.power_to_db(spectrogram, ref=np.max)
        libsora.display.specshow(spectrogram, sr=sample_rate, x_axis='time', y_axis='mel')

        plt.gcf().canvas.draw()
        audio_data = audio_data.reshape(fig.canvas.get_width_heigth()[::-1] + (3,))
        return (self.transforms(audio_data), label)

    def __len__(self):
        return self.length

In [ ]:
old_dataset = ESC50("ESC-50/train/")
start_time = time.process_time()
old_dataset.__getitem__(33)
end_time = tme.process_time()
old_time = end_time - start_time

new_dataset = ESC50Spectrogram("ESC-50/train/")
start_time = time.process_time()
new_dataset.__getitem__(33)
end_time = tme.process_time()
new_time = end_time - start_time

print(f"new time: {new_time}\nold time: {old_time}") # its one hundred times slower so we should use lru_cache

In [ ]:
import functools
class ESC50Spectrogram(Dataset):
    def __init__(self, path):
        files = path(path).glob("*.wav")
        # iterate through the listing and create tuple (filename,label)
        self.items = [f,f.split("-")[-1].replace(".wav","") for f in files]
        self.length = len(self.items)
        self.transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

    @functools.lru_cache(maxsize=<size of dataset>)
    def __getitem__(self,index):
        filename, label = self.items[index]
        audio_tensor, sample_rate = libsora.load(filename, sr=None)
        spectrogram = libsora.feature.melspectrogram(audio_tensor, sr=sample_rate)
        log_spectrogram = libsora.power_to_db(spectrogram, ref=np.max)
        libsora.display.specshow(log_spectrogram, sr=sample_rate, x_axis='time', y_axis='mel')

        plt.gcf().canvas.draw()
        audio_data = audio_data.reshape(fig.canvas.get_width_heigth()[::-1] + (3,))
        return (self.transforms(audio_data), label)

    def __len__(self):
        return self.length

my prefer approach is to precompute all possible plots and then create a new custom datset that load images from disk

In [ ]:
def precompute_spectrogram(path, dpi=50):
    files = Path(path).glob('*.wave')
    for filename in files:
        audio_tensor, sample_rate = libsora.load(filename, sr=None)
        log_spectrogram = libsora.power_to_db(spectrogram, ref=np.max)
        libsora.display.specshow(log_spectrogram, sr=sample_rate, x_axis='time', y_axis='mel')
        plt.gcf().savefig("{}{}_{}.png".format(filename.parant, dpi, filename.name),dpi=dpi)

In [ ]:
# we cant use ImageDataLoader because of filename but no matter we make our own
from PIL import ImageDataLoader

class precompute(Dataset):
    def __init__(self,path,dpi=50,transforms=None):
        files = Path(path).glob("{}*.wav.png".format(dpi))
        self.items = [(f, int(f.name.split("-")[-1].replace(".wave.png",""))) for f in files]
        self.length = len(self.items)
        if transforms=None:
            self.transforms = torchvision.transforms.Compose[(torchvision.transforms.ToTensor())]
        else:
            self.transforms = transforms
    
    def __getitem__(self, index):
        filename, label = self.items[index]
        img = Image.open(filename)
        return (self.transforms(img), label)
    
    def __len__(self):
        return self.length


In [ ]:
precompute_dataset = precompute("ESC-50/train/")
start_time = time.process_time()
precompute_dataset.__getitem__(33)
end_time = tme.process_time()
new_time = end_time - start_time

print(f"new time: {new_time}") # this one is as same as audio-base one

In [ ]:
# pretrained model
from torchvision import models
resnet = models.ResNet50(pretrained=True)

for param in resnet.parameters():
    param.requires_grad = false

resnet.fc = nn.Sequential(nn.Linear(resnet.fc.in_features,500),nn.ReLU(),nn.Dropout(), nn.Linear(500,50))

In [ ]:
# we also need to normalize the incoming images with the standard deviation and mean

esc50-train = PreparedESC50(PATH, transforms=torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
std=[0.229, 0.224, 0.225])]))

esc50-valid = PreparedESC50(PATH, transforms=torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
std=[0.229, 0.224, 0.225])]))

esc_train_loader = (esc50-train, bs, shuffle=True)
esc_valid_loader = (esc50-valid, bs, shuffle=True)

In [ ]:
# finding learning rate
resnet.save("resnet.pth")
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
logs, losses = find_lr(resnet, nn.CrossEntropyLoss(), optimizer)
plt.plot(logs,losses)

In [ ]:
optimizer = optim.Adam(resnet.parameters(), lr=[1e-2, 1e-4, 1e-8])
train(resnet, optimizer, nn.CrossEntropyLoss(), esc_train_loader, esc_valid_loader, epochs=5, device="cuda")